# Pandas

We'll breeze through the basics here, and get onto some assignments in a bit. I want to provide the *barest* of intuition so things stick down the road.

## Why pandas?

NumPy is great. But it lacks a few things that are conducive to doing statisitcal analysis. By building on top of NumPy, pandas provides

- labeled arrays
- heterogenous data types within a table
- "better" missing data handling
- convenient methods (`groupby`, `rolling`, `resample`)
- more data types (Categorical, Datetime)

## Data Structures

This is the typical starting point for any intro to pandas.
We'll follow suit.

### The DataFrame

Here we have the workhorse data structure for pandas.
It's an in-memory table holding your data, and provides a few conviniences over lists of lists or NumPy arrays.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Many ways to construct a DataFrame
# We pass a dict of {column name: column values}
np.random.seed(42)
df = pd.DataFrame({'A': [1, 2, 3], 
                   'B': [True, True, False],
                   'C': np.random.randn(3)},
                  index=['a', 'b', 'c'])  # also this weird index thing
df

Notice that we can store a column of intergers, a column of booleans, and a column of floats in the same `DataFrame`.

### Indexing

Our first improvement over numpy arrays is labeled indexing. We can select subsets by column, row, or both. Column selection uses the regular python `__getitem__` machinery. Pass in a single column label `'A'` or a list of labels `['A', 'C']` to select subsets of the original `DataFrame`.

In [ ]:
# Single column, reduces to a Series
df['A']

In [ ]:
cols = ['A', 'C']
df[cols]

For row-wise selection, use the special `.loc` accessor.

In [ ]:
df.loc[['a', 'b']]

You can use ranges to select rows or columns.

In [ ]:
df.loc['a':'b']

Notice that the slice is *inclusive* on both sides,  unlike your typical slicing of a list. Sometimes, you'd rather slice by *position* instead of label. `.iloc` has you covered:

In [ ]:
df.iloc[[0, 1]]

In [ ]:
df.iloc[:2]

This follows the usual python slicing rules: closed on the left, open on the right.

As I mentioned, you can slice both rows and columns. Use `.loc` for label or `.iloc` for position indexing.

In [ ]:
df.loc['a', 'B']

Pandas, like NumPy, will reduce dimensions when possible. Select a single column and you get back `Series` (see below). Select a single row and single column, you get a scalar.

You can get pretty fancy:

In [ ]:
df.loc['a':'b', ['A', 'C']]

#### Summary

- Use `[]` for selecting columns
- Use `.loc[row_lables, column_labels]` for label-based indexing
- Use `.iloc[row_positions, column_positions]` for positional index

I've left out boolean and hierarchical indexing, which we'll see later.

## Series

You've already seen some `Series` up above. It's the 1-dimensional analog of the DataFrame. Each column in a `DataFrame` is in some sense a `Series`. You can select a `Series` from a DataFrame in a few ways:

In [ ]:
# __getitem__ like before
df['A']

In [ ]:
# .loc, like before
df.loc[:, 'A']

In [ ]:
# using `.` attribute lookup
df.A

In [ ]:
df['mean'] = ['a', 'b', 'c']

In [ ]:
df['mean']

In [ ]:
df.mean

You'll have to be careful with the last one. It won't work if you're column name isn't a valid python identifier (say it has a space) or if it conflicts with one of the (many) methods on `DataFrame`. The `.` accessor is extremely convient for interactive use though.

You should never *assign* a column with `.` e.g. don't do

```python
# bad
df.A = [1, 2, 3]
```

It's unclear whether your attaching the list `[1, 2, 3]` as an attribute of `df`, or whether you want it as a column. It's better to just say

```python
df['A'] = [1, 2, 3]
# or
df.loc[:, 'A'] = [1, 2, 3]
```

`Series` share many of the same methods as `DataFrame`s.

## Index

`Index`es are something of a peculiarity to pandas.
First off, they are not the kind of indexes you'll find in SQL, which are used to help the engine speed up certain queries.
In pandas, `Index`es are about lables. This helps with selection (like we did above) and automatic alignment when performing operations between two `DataFrame`s or `Series`.

R does have row labels, but they're nowhere near as powerful (or complicated) as in pandas. You can access the index of a `DataFrame` or `Series` with the `.index` attribute.

In [ ]:
df.index

In [ ]:
df.columns

## Operations

In [ ]:
np.random.seed(42)
df = pd.DataFrame(np.random.uniform(0, 10, size=(3, 3)))
df

In [ ]:
df + 1

In [ ]:
df ** 2

In [ ]:
np.log(df)

DataFrames and Series have a bunch of useful aggregation methods, `.mean`, `.max`, `.std`, etc.

In [ ]:
df.mean()

## Loading Data

In [ ]:
df = pd.read_csv('beer_subset.csv.gz', parse_dates=['time'], compression='gzip')
review_cols = ['review_appearance', 'review_aroma', 'review_overall',
               'review_palate', 'review_taste']
df.head()

## Boolean indexing

Like a where clause in SQL. The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
df.abv < 5

In [ ]:
df[df.abv < 5].head()

Notice that we just used `[]` there. We can pass the boolean indexer in to `.loc` as well.

In [ ]:
df.loc[df.abv < 5, ['beer_style', 'review_overall']].head()

Again, you can get complicated

In [ ]:
df[((df.abv < 5) & (df.time > pd.Timestamp('2009-06'))) | (df.review_overall >= 4.5)]

<div class="alert alert-success">
    <b>Exercise</b>: Find the American beers
</div>

Select just the rows where the `beer_style` contains `'American'`. 

Hint: `Series` containing strings have a bunch of [useful methods](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) under the `DataFrame.<column>.str` namespace. Typically they correspond to regular python string methods, but

- They gracefully propogate missing values
- They're a bit more liberal about accepting regular expressions

We can't use `'American' in df['beer_style']`, since `in` is used to check membership in the series itself, not the strings. But `in` uses `__contains__`, so look for a string method like that.

In [ ]:
df.beer_style.str.contains("American")

In [ ]:
# Your solution
is_ipa = df.beer_style.str.contains("American")
df[is_ipa]

## Groupby

Groupby is a fundamental operation to pandas and data analysis.

The components of a groupby operation are to

1. Split a table into groups
2. Apply a function to each group
3. Combine the results

In pandas the first step looks like

```python
df.groupby( grouper )
```

`grouper` can be many things

- Series (or string indicating a column in `df`)
- function (to be applied on the index)
- dict : groups by *values*
- `levels=[ names of levels in a MultiIndex ]`

In [ ]:
gr = df.groupby('beer_style')
gr

Haven't really done anything yet. Just some book-keeping to figure out which **keys** go with which **rows**. Keys are the things we've grouped by (each `beer_style` in this case).

The last two steps, apply and combine, are just:

In [ ]:
gr.agg('mean')

In [ ]:
df.groupby('beer_style').mean()

This says apply the `mean` function to each column. Non-numeric columns (nusiance columns) are excluded. We can also select a subset of columns to perform the aggregation on.

In [ ]:
gr[review_cols].agg('mean')

`.` attribute lookup works as well.

In [ ]:
gr.abv.agg('mean')

Certain operations are attached directly to the `GroupBy` object, letting you bypass the `.agg` part

In [ ]:
gr.abv.mean()

Now we'll run the gamut on a bunch of grouper / apply combinations.
Keep sight of the target though: split, apply, combine.

- Grouper: Controls the output index
    * single grouper -> `Index`
    * array-like grouper -> `MultiIndex`
- Subject (Groupee): Controls the output data values
    * single column -> `Series` (or DataFrame if multiple aggregations)
    * multiple columns -> `DataFrame`
- Aggregation: Controls the output columns
    * single aggfunc -> `Index` in the colums
    * multiple aggfuncs -> `MultiIndex` in the columns (Or 1-D Index groupee is 1-d)

Multiple Aggregations on one column

In [ ]:
gr['review_aroma'].agg(['mean', 'std', 'count']).head()

Single Aggregation on multiple columns

In [ ]:
gr[review_cols].mean()

Multiple aggregations on multiple columns

In [ ]:
gr[review_cols].agg(['mean', 'count', 'std'])

Hierarchical Indexes in the columns can be awkward to work with, so I'll usually
move a level to the Index with `.stack`.

In [ ]:
multi = gr[review_cols].agg(['mean', 'count', 'std']).stack(level=0)
multi.head(10)

You can group by **levels** of a MultiIndex.

In [ ]:
multi.groupby(level='beer_style')['mean'].agg(['min', 'max'])

Group by **multiple** columns

In [ ]:
df.groupby(['brewer_id', 'beer_style']).review_overall.mean()

In [ ]:
df.groupby(['brewer_id', 'beer_style'])[review_cols].mean()